In [1]:
import os
import cv2
import numpy as np

In [2]:
origin_dir = 'origin-Si-smo2-128'
source_dir = 'results-Si-smo2-128'
target_dir = 'stitched-Si-smo2-128'

os.makedirs(origin_dir, exist_ok=True)
os.makedirs(source_dir, exist_ok=True)
os.makedirs(target_dir, exist_ok=True)

origin_files = os.listdir(origin_dir)
source_files_all = os.listdir(source_dir)

In [5]:
def distance_matrix(min_value = 0.01, size=256, channel=3):
    """
    Create a square matrix with elements linearly decaying from the center.
    
    Args:
    - min_value (float): The minimum value at the edges of the matrix.
    - size (int): The size of the matrix (must be a positive integer).
    - channel(int): The number of channel(s) (mmust be a positive integer).
    Returns:
    - np.ndarray: The generated matrix.
    """
    if size <= 0:
        raise ValueError("Size must be a positive integer")
    if channel <= 0:
        raise ValueError("Channel must be a positive integer")
    
    matrix = np.zeros((size, size, channel), dtype=float)

    # Calculate the center of the matrix
    center = (size - 1) / 2
    
    # Calculate the maximum distance from the center
    max_distance = np.sqrt(center**2 + center**2)

    for i in range(size):
        for j in range(size):
            distance = np.sqrt((i - center)**2 + (j - center)**2)
            value = 1 - (distance / max_distance) * (1 - min_value)
            for k in range(channel):
                matrix[i, j, k] = value

    return matrix

def stitch(nx, ny, sx, sy, cuts):
    """_summary_
    Modified stitch function for better performance while processing larger image

    Args:
        nx (_type_): images size
        ny (_type_): image size
        sx (_type_): stride size
        sy (_type_): stride size
        cuts (_type_): cuts of image

    Returns:
        _type_: _description_
    """

    # lx, ly: the cut size
    (_, lx, ly, num_class) = np.shape(cuts)

    x_list = np.arange(0, nx - lx + 1, sx)
    y_list = np.arange(0, ny - ly + 1, sy)
    if x_list[-1]+lx < nx:
        x_list = np.append(x_list, nx-lx)
    if y_list[-1]+ly < ny:
        y_list = np.append(y_list, ny-ly)

    Nx = len(x_list)
    final_img = np.zeros((nx, ny, num_class))
    cnt = np.zeros((nx, ny, num_class))
    dist_coef = distance_matrix(min_value=0.01, size=lx, channel=num_class)
    for idx, img in enumerate(cuts):
        i = idx % Nx
        j = idx //Nx
        final_img[x_list[i]:x_list[i] + lx,  y_list[j]:y_list[j] + ly, :] += np.multiply(img, dist_coef)   
        cnt[x_list[i]:x_list[i] + lx,  y_list[j]:y_list[j] + ly,:] += dist_coef   
        # final_img[x_list[i]:x_list[i] + lx,  y_list[j]:y_list[j] + ly, :] += img   
        # cnt[x_list[i]:x_list[i] + lx,  y_list[j]:y_list[j] + ly,:] += 1
    return final_img/cnt

In [6]:
for origin in origin_files:
    ori_dir = os.path.join(origin_dir, origin)
    ori_img = cv2.imread(ori_dir)
    ori_img = np.array(ori_img)
    (size_x, size_y, _) = ori_img.shape
    
    (lx, ly) = (256, 256)
    (sx, sy) = (128, 128) 
    
    source_files = [x for x in source_files_all if x[:x.rfind("_")]==origin[:origin.find(".")]]
    print(size_x, size_y)
    source_files.sort(key=lambda x: int(x[x.rfind('_')+1:x.find('.')]))
    print(source_files)
    cuts = np.zeros((len(source_files), 256, 256, 3), dtype=np.uint8)
    count = 0
    for file in source_files:
        pic_dir = os.path.join(source_dir, file)
        input_img = cv2.imread(pic_dir)
        input_img = cv2.cvtColor(input_img,cv2.COLOR_BGR2GRAY)
        input_img = np.array(input_img, dtype=np.uint8)
        cuts[count][:, :, 0] = input_img
        cuts[count][:, :, 1] = input_img
        cuts[count][:, :, 2] = input_img
        count += 1
    print(cuts.shape)

    stitched = stitch(size_x, size_y, sx, sy, cuts)

    save_dir = os.path.join(target_dir, origin)
    cv2.imwrite(save_dir, stitched)

1024 1024
['P4_x15M_250mm_39_DFUpper_0.png', 'P4_x15M_250mm_39_DFUpper_1.png', 'P4_x15M_250mm_39_DFUpper_2.png', 'P4_x15M_250mm_39_DFUpper_3.png', 'P4_x15M_250mm_39_DFUpper_4.png', 'P4_x15M_250mm_39_DFUpper_5.png', 'P4_x15M_250mm_39_DFUpper_6.png', 'P4_x15M_250mm_39_DFUpper_7.png', 'P4_x15M_250mm_39_DFUpper_8.png', 'P4_x15M_250mm_39_DFUpper_9.png', 'P4_x15M_250mm_39_DFUpper_10.png', 'P4_x15M_250mm_39_DFUpper_11.png', 'P4_x15M_250mm_39_DFUpper_12.png', 'P4_x15M_250mm_39_DFUpper_13.png', 'P4_x15M_250mm_39_DFUpper_14.png', 'P4_x15M_250mm_39_DFUpper_15.png', 'P4_x15M_250mm_39_DFUpper_16.png', 'P4_x15M_250mm_39_DFUpper_17.png', 'P4_x15M_250mm_39_DFUpper_18.png', 'P4_x15M_250mm_39_DFUpper_19.png', 'P4_x15M_250mm_39_DFUpper_20.png', 'P4_x15M_250mm_39_DFUpper_21.png', 'P4_x15M_250mm_39_DFUpper_22.png', 'P4_x15M_250mm_39_DFUpper_23.png', 'P4_x15M_250mm_39_DFUpper_24.png', 'P4_x15M_250mm_39_DFUpper_25.png', 'P4_x15M_250mm_39_DFUpper_26.png', 'P4_x15M_250mm_39_DFUpper_27.png', 'P4_x15M_250mm_39_D